In [1]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='pnc', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pnc...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:24


In [45]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


Console
- 6.16 ~ 6.29

In [20]:
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

user = load_data_mart("console", "2022-06-29", "2022-06-29", "user_master").where(col('lastlogindate') >= "2022-06-16")
user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left")

In [21]:
gcoin = load_data_mart("console", "2022-06-16", "2022-06-29", "gcoin_use").where(col("event_name").like("%pnc%"))
user = user.withColumnRenamed("accountid", "account_id")
membership = load_membership("pubg", "console", "2022-06-29", "2022-06-29")
user = user.join(membership, "account_id")

In [22]:
pnc = gcoin.join(user, "account_id", "left")

In [23]:
region_au = user.groupBy("pubg_region").agg(countDistinct("account_id").alias("au"))

In [24]:
region_pu = pnc.groupBy("pubg_region").agg(countDistinct("account_id").alias("pu"), sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use"))

In [25]:
region_paid_pu = pnc.where(col("paid_use") > 0).groupBy("pubg_region").agg(countDistinct("account_id").alias("paid_pu"))

In [26]:
region_pu = region_pu.join(region_paid_pu, "pubg_region")

In [27]:
region_kpi = region_au.join(region_pu, "pubg_region").withColumn("pur", col("pu")/col("au")).withColumn("paid_pur", col("paid_pu")/col("au")).withColumn("total_sales", (col("free_use") + col("paid_use"))/100).withColumn("paid_sales", col("paid_use")/100).toPandas()

In [28]:
region_kpi

,pubg_region,au,pu,free_use,paid_use,paid_pu,pur,paid_pur,total_sales,paid_sales
0,CN,8662,18,6180,11740,13,0.002078,0.001501,179.2,117.4
1,NA,416695,4796,1236470,3290830,3505,0.011510,0.008411,45273.0,32908.3
2,SA,137740,871,321090,521140,544,0.006324,0.003949,8422.3,5211.4
3,KR,3837,44,12550,36210,35,0.011467,0.009122,487.6,362.1
4,SEA,15799,37,11980,26840,25,0.002342,0.001582,388.2,268.4
5,JP,13814,177,44770,159270,143,0.012813,0.010352,2040.4,1592.7
6,TW/HK,3329,23,6130,13800,17,0.006909,0.005107,199.3,138.0
7,CIS,29982,84,64200,20920,21,0.002802,0.000700,851.2,209.2
8,EMEA,228053,1319,421780,886470,902,0.005784,0.003955,13082.5,8864.7
9,OC,18432,256,53460,206940,204,0.013889,0.011068,2604.0,2069.4
